# Import required libraries

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection settings

In [7]:
DB_USER = "root"
DB_PASS = "root"
DB_HOST = "localhost"  # Use 'localhost' if running Jupyter on your host
DB_PORT = "5432"
DB_NAME = "nyc_taxi"

# Create connection engine

In [8]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Trip segmentation SQL query

In [13]:
query = """
SELECT
  COUNT(*) FILTER (WHERE trip_distance <= 1) AS "Up to 1 mile",
  COUNT(*) FILTER (WHERE trip_distance > 1 AND trip_distance <= 3) AS "1–3 miles",
  COUNT(*) FILTER (WHERE trip_distance > 3 AND trip_distance <= 7) AS "3–7 miles",
  COUNT(*) FILTER (WHERE trip_distance > 7 AND trip_distance <= 10) AS "7–10 miles",
  COUNT(*) FILTER (WHERE trip_distance > 10) AS "Over 10 miles"
FROM yellow_taxi_data
WHERE tpep_pickup_datetime >= '2021-01-01'
  AND tpep_pickup_datetime < '2021-03-01';
"""

# Run the query and load into DataFrame

In [14]:
df = pd.read_sql(query, engine)

# Show results

In [15]:
print("Trip segmentation from October 2021:")
display(df)

Trip segmentation from October 2021:


,Up to 1 mile,1–3 miles,3–7 miles,7–10 miles,Over 10 miles
0,353595,672030,223943,48211,71966


Longest trip for each day:
Find the single trip with the maximum distance per pickup day in January 2021, and list the pickup days with the longest of those maximums.

In [16]:
query = """
SELECT 
    DATE(tpep_pickup_datetime) AS pickup_date,
    MAX(trip_distance) AS max_trip_distance
FROM yellow_taxi_data
WHERE tpep_pickup_datetime >= '2021-01-01' AND tpep_pickup_datetime < '2021-02-01'
GROUP BY pickup_date
ORDER BY max_trip_distance DESC
LIMIT 10;
"""

Show results

In [18]:
df = pd.read_sql(query, engine)
display(df)

,pickup_date,max_trip_distance
0,2021-01-29,263163.28
1,2021-01-28,140549.58
2,2021-01-31,114328.20
3,2021-01-13,112257.10
4,2021-01-11,85947.42
5,2021-01-03,84813.76
6,2021-01-14,75167.82
7,2021-01-26,65923.66
8,2021-01-20,51964.09
9,2021-01-19,51848.62


In [21]:
# SQL Query
query = """
SELECT 
    z."Zone" AS pickup_zone,
    SUM(ytd.total_amount) AS total_revenue
FROM yellow_taxi_data ytd
JOIN zones z ON ytd."PULocationID" = z."LocationID"
WHERE DATE(ytd.tpep_pickup_datetime) = '2021-01-18'
GROUP BY pickup_zone
HAVING SUM(ytd.total_amount) > 13000
ORDER BY total_revenue DESC;
"""

In [23]:
df = pd.read_sql(query, engine)
df

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "zones" does not exist
LINE 6: JOIN zones z ON ytd."PULocationID" = z."LocationID"
             ^

[SQL: 
SELECT 
    z."Zone" AS pickup_zone,
    SUM(ytd.total_amount) AS total_revenue
FROM yellow_taxi_data ytd
JOIN zones z ON ytd."PULocationID" = z."LocationID"
WHERE DATE(ytd.tpep_pickup_datetime) = '2021-01-18'
GROUP BY pickup_zone
HAVING SUM(ytd.total_amount) > 13000
ORDER BY total_revenue DESC;
]
(Background on this error at: https://sqlalche.me/e/20/f405)